# 240916
1. Regression 기준을 바꿔야하나 고민.  X_reg값으로 regression하면, PointMRC, PSM Input은  제외됨. 
2. X_reg 는 순수raw값임. 계측된 결과만 가지고 분석하겠다는 의미임. 
3. Point MRC는 빠져도 되나 ?   M3S에 포함되나?? 
   OCM에서 Regression할때에 Point MRC도 더해주고나서 regress함. -> rk input 커지게하려는  apc제어 목적임.
   분석목적에서는 넣을지 뺄지 잘 모르겠음....
   Point MRC가 엄청 큰 경우에, ocm map에선 보일것임. 근데 그게 의미가 있나 ? 실제계측이랑은 차이가 있는데. 
   ocm에서는 제어목적으로 순수raw + MRC_RX(point mrc)를 더해준것으로 F1을 Regression해야되서 필요한게 맞음. 
   계측값으로 분석하는 경우에는 뺴는게 맞을거같고.  필요하면 넣는것도 추가하던지해야겠음


   



In [1]:
import pandas as pd
import numpy as np

# 엑셀 파일에서 데이터 불러오기
df = pd.read_excel("C:/vscode/MODULE/output.xlsx")




# die x, die y, step pitch x, step pitch y 값 추출
die_x = df['DieX']
die_y = df['DieY']
step_pitch_x = df['STEP_PITCH_X']
step_pitch_y = df['STEP_PITCH_Y']
map_shift_x = df['MAP_SHIFT_X']
map_shift_y = df['MAP_SHIFT_Y']
field_x = df['coordinate_X']
field_y = df['coordinate_Y']

# x, y 좌표 계산
x = die_x * step_pitch_x + map_shift_x
y = die_y * step_pitch_y + map_shift_y
rx = field_x
ry = field_y


'''
RK는 1023/511 모드임. RK20제거
'''


# X_dx 데이터 구성 (wk5 계산 시만 -y 사용, 3차항 포함)
X_dx = np.vstack([
    np.ones(len(x)), (x)/10**6, (-y)/10**6, (x**2)/10**12, (x*y)/10**12, (y**2)/10**12, (x**3)/10**15, (x**2*y)/10**15, (x*y**2)/10**15, (y**3)/10**15, 
                     (rx)/10**6, (-ry)/10**6, (rx**2)/10**9, (rx*ry)/10**9, (ry**2)/10**9, (rx**3)/10**12, (rx**2*ry)/10**12, (rx*ry**2)/10**12, (ry**3)/10**12
]).T


# X_dy 데이터 구성 (3차항 포함, y를 그대로 사용)
X_dy = np.vstack([
    np.ones(len(y)), (y)/10**6, (x)/10**6, (y**2)/10**12, (y*x)/10**12, (x**2)/10**12, (y**3)/10**15, (y**2*x)/10**15, (y*x**2)/10**15, (x**3)/10**15,
                     (ry)/10**6, (rx)/10**6, (ry**2)/10**9, (ry*rx)/10**9, (rx**2)/10**9, (ry**3)/10**12, (ry**2*rx)/10**12, (ry*rx**2)/10**12
]).T



# Y_dx, Y_dy 데이터
Y_dx = df['X_reg']
Y_dy = df['Y_reg']

# 최소자승법으로 dx의 계수 구하기 (X_dx 사용)
coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]

# 최소자승법으로 dy의 계수 구하기 (X_dy 사용)
coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]




# dx의 결과 출력 (3차항 포함 계수명 추가)
# dy의 결과 출력 (3차항 포함 계수명 추가)

print("Coefficients for misalign x(um):")
print("\nCoefficients for misalign y(um):")

print(f"WK1 (Offset_x): {coeff_dx[0]}")
print(f"WK2 (Offset_y): {coeff_dy[0]}")
print(f"WK3 (Scale_x): {coeff_dx[1]}")
print(f"WK4 (Scale_y): {coeff_dy[1]}")
print(f"WK5 (Rotation_x_with_-y): {coeff_dx[2]}")  # 여기서 -y 반영됨
print(f"WK6 (Rotation_y): {coeff_dy[2]}")
print(f"WK7 (x^2): {coeff_dx[3]}")
print(f"WK8 (x^2): {coeff_dy[3]}")
print(f"WK9 (xy): {coeff_dx[4]}")
print(f"WK10 (xy): {coeff_dy[4]}")
print(f"WK11 (y^2): {coeff_dx[5]}")
print(f"WK12 (y^2): {coeff_dy[5]}")
print(f"WK13 (x^3): {coeff_dx[6]}")
print(f"WK14 (x^3): {coeff_dy[6]}")
print(f"WK15 (x^2*y): {coeff_dx[7]}")
print(f"WK16 (x^2*y): {coeff_dy[7]}")
print(f"WK17 (x*y^2): {coeff_dx[8]}")
print(f"WK18 (x*y^2): {coeff_dy[8]}")
print(f"WK19 (y^3): {coeff_dx[9]}")
print(f"WK20 (y^3): {coeff_dy[9]}")


print(f"RK1 (Offset_x): ")
print(f"RK2 (Offset_y): ")

print(f"RK3 (Scale_x): {coeff_dx[10]}")
print(f"RK4 (Scale_y): {coeff_dy[10]}")
print(f"RK5 (Rotation_x_with_-y): {coeff_dx[11]}")  # 여기서 -y 반영됨
print(f"RK6 (Rotation_y): {coeff_dy[11]}")
print(f"RK7 (x^2): {coeff_dx[12]}")
print(f"RK8 (x^2): {coeff_dy[12]}")
print(f"RK9 (xy): {coeff_dx[13]}")
print(f"RK10 (xy): {coeff_dy[13]}")
print(f"RK11 (y^2): {coeff_dx[14]}")
print(f"RK12 (y^2): {coeff_dy[14]}")
print(f"RK13 (x^3): {coeff_dx[15]}")
print(f"RK14 (x^3): {coeff_dy[15]}")
print(f"RK15 (x^2*y): {coeff_dx[16]}")
print(f"RK16 (x^2*y): {coeff_dy[16]}")
print(f"RK17 (x*y^2): {coeff_dx[17]}")
print(f"RK18 (x*y^2): {coeff_dy[17]}")
print(f"RK19 (y^3): {coeff_dx[18]}")








Coefficients for misalign x(um):

Coefficients for misalign y(um):
WK1 (Offset_x): 0.00041469869714542095
WK2 (Offset_y): -0.0005026897023256345
WK3 (Scale_x): -0.004418772622024333
WK4 (Scale_y): -0.007466385037138512
WK5 (Rotation_x_with_-y): -8.137266471773949e-05
WK6 (Rotation_y): -0.002040200099101792
WK7 (x^2): -0.006361246638125382
WK8 (x^2): 0.008042167207290845
WK9 (xy): 0.009198953218054015
WK10 (xy): 0.0017450323980699627
WK11 (y^2): 0.01553386957231498
WK12 (y^2): -0.009766600054838306
WK13 (x^3): -0.0007120860401055387
WK14 (x^3): -0.00046310268766206256
WK15 (x^2*y): -4.931365001857575e-05
WK16 (x^2*y): 0.00010575771323497
WK17 (x*y^2): -0.0002828240637442234
WK18 (x*y^2): -0.00033400331323841164
WK19 (y^3): 0.00010961360306467575
WK20 (y^3): 0.0001052128612540279
RK1 (Offset_x): 
RK2 (Offset_y): 
RK3 (Scale_x): 0.049978641119455526
RK4 (Scale_y): 0.004022623702154364
RK5 (Rotation_x_with_-y): 0.003133372959516833
RK6 (Rotation_y): -0.04548861601128858
RK7 (x^2): -0.00430

In [17]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook


# 엑셀 파일에서 데이터 불러오기
df = pd.read_excel("C:/vscode/MODULE/output.xlsx")

# die x, die y, step pitch x, step pitch y 값 추출
die_x = df['DieX']
die_y = df['DieY']
step_pitch_x = df['STEP_PITCH_X']
step_pitch_y = df['STEP_PITCH_Y']
map_shift_x = df['MAP_SHIFT_X']
map_shift_y = df['MAP_SHIFT_Y']
field_x = df['coordinate_X']
field_y = df['coordinate_Y']

# x, y 좌표 계산
x = die_x * step_pitch_x + map_shift_x
y = die_y * step_pitch_y + map_shift_y
rx = field_x
ry = field_y

# X_dx 데이터 구성 (wk5 계산 시만 -y 사용, 3차항 포함)
X_dx = np.vstack([
    np.ones(len(x)), (x)/10**6, (-y)/10**6, (x**2)/10**12, (x*y)/10**12, (y**2)/10**12, (x**3)/10**15, (x**2*y)/10**15, (x*y**2)/10**15, (y**3)/10**15, 
                     (rx)/10**6, (-ry)/10**6, (rx**2)/10**9, (rx*ry)/10**9, (ry**2)/10**9, (rx**3)/10**12, (rx**2*ry)/10**12, (rx*ry**2)/10**12, (ry**3)/10**12
]).T

# X_dy 데이터 구성 (3차항 포함, y를 그대로 사용)
X_dy = np.vstack([
    np.ones(len(y)), (y)/10**6, (x)/10**6, (y**2)/10**12, (y*x)/10**12, (x**2)/10**12, (y**3)/10**15, (y**2*x)/10**15, (y*x**2)/10**15, (x**3)/10**15,
                     (ry)/10**6, (rx)/10**6, (ry**2)/10**9, (ry*rx)/10**9, (rx**2)/10**9, (ry**3)/10**12, (ry**2*rx)/10**12, (ry*rx**2)/10**12
]).T

# Y_dx, Y_dy 데이터
Y_dx = df['X_reg']
Y_dy = df['Y_reg']

# 최소자승법으로 dx의 계수 구하기 (X_dx 사용)
coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]

# 최소자승법으로 dy의 계수 구하기 (X_dy 사용)
coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]









# 길이가 다를 경우 자동으로 NaN을 추가해 DataFrame 생성
coeff_df = pd.DataFrame({
    'Coeff_dx': pd.Series(coeff_dx),
    'Coeff_dy': pd.Series(coeff_dy)
})




# 기존 파일 불러오기
file_path = "C:/vscode/MODULE/output.xlsx"
sheet_name = "OSR(WK_RK)"

# 기존 파일 불러오기
book = load_workbook(file_path)

# 기존 시트가 있을 경우 삭제
if sheet_name in book.sheetnames:
    del book[sheet_name]  # 기존 시트 삭제

# 새로운 시트에 값 저장
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    writer.book = book  # 기존 파일 내용을 가져오기
    coeff_df.to_excel(writer, sheet_name=sheet_name, index=False)
    





C:\Users\oth08\AppData\Local\Temp\ipykernel_14280\813194052.py:71: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book  # 기존 파일 내용을 가져오기
